<a href="https://colab.research.google.com/github/gnoparus/bualabs/blob/master/nbs/32a_langchain_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ต่อไปเราจะมาเรียนรู้ LangChain

# 0. Magic Commands
ให้ใส่ไว้บนสุดทุก Notebook เป็นการสั่งให้ Notebook ก่อนรัน ให้รีโหลด Library ภายนอกที่เรา import ไว้ใหม่โดยอัตโนมัติ

และให้พล็อตกราฟ matplotlib ใน Output ของ cell แบบ code ได้เลย

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

สั่งให้ Colab แสดงผลข้อความแบบ Wrap Text

In [2]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# 1. Import Library

ติดตั้ง OpenAI Library ด้วยการ Uncomment Cell ด้านล่างแล้ว Run

In [76]:
# ! pip install openai

# 2. Authentication

ก่อนที่เราจะเรียกใช้งาน ChatGPT, GPT-4 ของ OpenAI ได้ เราต้องลงทะเบียน สมัครใช้งาน และใส่ข้อมูลวิธีชำระเงินให้เรียบร้อยก่อน 

## Sign up

กดปุ่ม Sign up สีเขียว ในลิงค์ด้านล่าง สมัครใช้งาน OpenAI 

https://platform.openai.com/overview

ใส่ข้อมูลบัตรเครดิต ในหน้า Billing ให้เรียบร้อย

https://platform.openai.com/account/billing/payment-methods






## API Key

สร้าง API Key จากลิงค์ด้านล่าง โดยกดปุ่ม Create new secret key ตั้งชื่อ แล้ว Copy secret key เตรียมไว้ใส่ในช่อง OPENAI_API_KEY ด้านล่าง

https://platform.openai.com/account/api-keys


## ตัวอย่าง API Key 

OPENAI_API_KEY: sk-abcDeFGHijKlMNopQrstuvWxyZ1234567890abcDefhiJklm

รัน Cell ด้านล่าง แล้วใส่ secret key ที่ได้มา

บางที Run แล้วค้าง ไม่ขึ้นช่องให้กรอก ให้กดหยุด แล้วลอง Run ใหม่

In [6]:
import os
from getpass import getpass

token = getpass('Enter OPENAI_API_KEY here: ')
# print ('token is', token)

os.environ["OPENAI_API_KEY"] = token

Enter OPENAI_API_KEY here: ··········


หลังจากทดสอบเสร็จแล้ว แนะนำให้กลับไปลบ Secret Key ในหน้า https://platform.openai.com/account/api-keys ทิ้งไปทุกครั้ง เพื่อความปลอดภัย โดยเราสามารถสร้าง Key ใหม่ได้เรื่อย ๆ ไม่จำกัด

#3. Call API 

## 3.1 เรียกใช้ OpenAI API แบบ ตรง ๆ 

[จาก ep ก่อน Prompt Engineering คืออะไร ](https://www.bualabs.com/archives/4483/what-is-prompt-engineering/) เราได้เรียกใช้ chatgpt ผ่าน API ของ OpenAI ได้โดยตรง

In [7]:
import openai

In [8]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]


In [20]:
country = "Thailand"
language = "English"

prompt31 = f"""Give me reasons why {country} is a good place to retire: \
Cost of living, Healthcare, Climate, Safety, Lifestyle, and Visa. \
Output in {language}.
"""

In [21]:
print(prompt31)

Give me reasons why Thailand is a good place to retire: Cost of living, Healthcare, Climate, Safety, Lifestyle, and Visa. Output in English.



In [22]:
reason31 = get_completion(prompt31)

In [23]:
print(reason31)

Thailand is a popular destination for retirees due to several reasons. Firstly, the cost of living in Thailand is relatively low compared to other countries, making it an affordable option for retirees on a fixed income. Secondly, Thailand has a well-developed healthcare system with modern facilities and highly trained medical professionals. The cost of healthcare is also significantly lower than in many Western countries.

Thirdly, Thailand has a warm and tropical climate, which is ideal for retirees who want to enjoy a relaxed and comfortable lifestyle. The country has beautiful beaches, lush forests, and a rich cultural heritage, making it an attractive destination for retirees who want to explore and experience new things.

Fourthly, Thailand is generally considered a safe country, with low crime rates and a friendly and welcoming population. Retirees can feel secure and comfortable living in Thailand, knowing that they are in a safe and supportive environment.

Fifthly, Thailand o

In [24]:
country = "Singapore"
language = "English with stutter in every sentences"

prompt31 = f"""Give me reasons why {country} is a good place to retire: \
Cost of living, Healthcare, Climate, Safety, Lifestyle, and Visa. \
Output in {language}.
"""

In [25]:
print(prompt31)

Give me reasons why Singapore is a good place to retire: Cost of living, Healthcare, Climate, Safety, Lifestyle, and Visa. Output in English with stutter in every sentences.



In [26]:
reason31 = get_completion(prompt31)
print(reason31)

W-well, S-Singapore is a g-great place to retire because of its c-cost of living. It may be a bit h-higher than other countries in the region, but it's still r-reasonable compared to other developed countries. And the h-healthcare system is top-notch, with w-world-class hospitals and medical facilities. The c-climate is also a plus, with a t-tropical climate that's warm and sunny all year round. And let's not forget about the s-safety. Singapore is one of the safest countries in the world, with l-low crime rates and strict laws. The l-lifestyle is also great, with plenty of activities and events to keep you busy. And if you're a f-foreigner, getting a visa to retire in Singapore is relatively easy, with a number of options available. So, all in all, Singapore is a g-great place to retire for those who value quality of life and peace of mind.


## 3.2 เรียกใช้ ChatGPT ผ่าน LangChain

เราสามารถใช้ F-String ในการสร้าง Template เพื่อสร้าง Prompt ส่งไปยัง ChatGPT ได้ไม่มีปัญหา แต่ถ้าเป็น Prompt ที่ซับซ้อนขึ้น มีหลาย Step และมี Output เป็นโครงสร้าง ที่จะนำไปใช้ต่อเช่น JSON การใช้คลาสเฉพาะ เช่น ChatPromptTemplate ของ LangChain สะดวกกว่า

In [27]:
### https://python.langchain.com/en/latest/modules/models/llms/integrations.html

In [75]:
# ! pip install langchain

In [29]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI


In [30]:
chat = ChatOpenAI(temperature=0.0)
chat

ChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key=None, openai_api_base=None, openai_organization=None, openai_proxy=None, request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None)

ChatPromptTemplate

In [31]:
template_string32 = """Give me reasons why {country} is a good place to retire: \
Cost of living, Healthcare, Climate, Safety, Lifestyle, and Visa. \
Output in {language}.
"""

In [32]:
prompt_template32 = ChatPromptTemplate.from_template(template_string32)

In [33]:
# prompt_template32

In [34]:
# prompt_template32.messages

In [35]:
# prompt_template32.messages[0]

In [36]:
prompt_template32.messages[0].prompt

PromptTemplate(input_variables=['country', 'language'], output_parser=None, partial_variables={}, template='Give me reasons why {country} is a good place to retire: Cost of living, Healthcare, Climate, Safety, Lifestyle, and Visa. Output in {language}.\n', template_format='f-string', validate_template=True)

In [37]:
country_to_retire = "Japanese"   # Not Japan
language_to_write = "Rude English"

prompt32 = prompt_template32.format_messages(
                    country=country_to_retire,
                    language=language_to_write)

In [38]:
print(type(prompt32))
print(prompt32)

<class 'list'>
[HumanMessage(content='Give me reasons why Japanese is a good place to retire: Cost of living, Healthcare, Climate, Safety, Lifestyle, and Visa. Output in Rude English.\n', additional_kwargs={}, example=False)]


In [39]:
prompt32[0]

HumanMessage(content='Give me reasons why Japanese is a good place to retire: Cost of living, Healthcare, Climate, Safety, Lifestyle, and Visa. Output in Rude English.\n', additional_kwargs={}, example=False)

In [40]:
reasons32 = chat(prompt32)

In [41]:
print(reasons32.content)

1. Cost of living is f\*cking cheap in Japan, so you can stretch your retirement savings like a rubber band.
2. Healthcare is top-notch, so you won't have to worry about getting sick and dying broke.
3. The climate is pretty sweet, with mild winters and warm summers, so you can enjoy the outdoors all year round.
4. Japan is one of the safest countries in the world, so you won't have to worry about getting mugged or murdered on your way to the grocery store.
5. The lifestyle is pretty chill, with lots of opportunities for cultural experiences, outdoor activities, and delicious food.
6. Getting a visa to retire in Japan is relatively easy, so you won't have to jump through a bunch of hoops to make it happen.


# 4. Output Format

ใน [ep ก่อน เรื่องระบุรูปแบบโครงสร้างข้อมูลผลลัพท์](https://www.bualabs.com/archives/4703/how-to-prompt-chatgpt-format-output-prompt-engineering-ep-2/) เราได้พูดถึงเรื่อง การกำหนด Format โครงสร้างผลลัพท์ ที่ต้องการใน Prompt เพื่อนำไปใช้ต่อ

## 4.1 ระบุโครงสร้างผลลัพท์ ที่ต้องการใน Prompt โดยตรง

In [43]:
template_string41 = """Give me reasons why {country} is a good place to retire: \
Cost of living, Healthcare, Climate, Safety, Lifestyle, and Visa. \
Write in {language}. Output in JSON Format.
"""

In [44]:
prompt_template41 = ChatPromptTemplate.from_template(template_string41)

In [45]:
country_to_retire = "Mexico"   # Not Japan
language_to_write = "Polite English"

prompt41 = prompt_template41.format_messages(
                    country=country_to_retire,
                    language=language_to_write)

In [46]:
reasons41 = chat(prompt41)

In [47]:
print(reasons41.content)

{
    "Cost of living": "Mexico has a lower cost of living compared to many other countries, making it an affordable option for retirees.",
    "Healthcare": "Mexico has a good healthcare system, with many hospitals and clinics offering high-quality care at a fraction of the cost of other countries.",
    "Climate": "Mexico has a warm and sunny climate, making it an ideal place for retirees who enjoy spending time outdoors.",
    "Safety": "While there are some safety concerns in certain areas of Mexico, overall it is a safe place to retire. Many retirees feel safe and secure living in Mexico.",
    "Lifestyle": "Mexico has a rich culture and history, with many opportunities for retirees to explore and enjoy. The country is also known for its delicious food, beautiful beaches, and vibrant nightlife.",
    "Visa": "Mexico offers a retirement visa program, which makes it easy for retirees to obtain residency and enjoy all the benefits of living in the country."
}


In [48]:
print(type(reasons41.content))

<class 'str'>


## 4.2 ใช้ StructuredOutputParser และ ResponseSchema จาก LangChain

In [49]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [55]:
col_schema = ResponseSchema(name="cost_of_living",
                             description="Assess the cost of housing, healthcare, groceries, transportation, and other essential expenses in the country you are considering. Ensure that your retirement income or savings can adequately cover your desired lifestyle.")
healthcare_schema = ResponseSchema(name="healthcare",
                                      description="Look into the quality and accessibility of healthcare services in the country. Consider the availability of affordable healthcare insurance or the country's public healthcare system.")
climate_schema = ResponseSchema(name="climate",
                                      description="Consider the climate and natural surroundings of the country. Determine if you prefer a warm or cold climate, proximity to the beach, mountains, or other natural attractions.")
safety_schema = ResponseSchema(name="safety",
                                      description="Research the country's safety record, crime rates, and political stability. Consider how safe you will feel living there and if the country has a stable legal system that protects your rights.")
lifestyle_schema = ResponseSchema(name="lifestyle",
                                      description="Evaluate whether the country's culture, language, and social environment align with your preferences. Consider the availability of recreational activities, entertainment, and amenities that cater to your interests.")
visa_schema = ResponseSchema(name="visa",
                                      description="Understand the visa and residency requirements for retirees in the country you are considering. Determine if you are eligible for retirement visas or long-term residency, and consider the ease of obtaining such permits.")

reasons_schemas = [col_schema, healthcare_schema, climate_schema, safety_schema, lifestyle_schema, visa_schema]

In [56]:
output_parser = StructuredOutputParser.from_response_schemas(reasons_schemas)

In [57]:
format_instructions = output_parser.get_format_instructions()

In [58]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"cost_of_living": string  // Assess the cost of housing, healthcare, groceries, transportation, and other essential expenses in the country you are considering. Ensure that your retirement income or savings can adequately cover your desired lifestyle.
	"healthcare": string  // Look into the quality and accessibility of healthcare services in the country. Consider the availability of affordable healthcare insurance or the country's public healthcare system.
	"climate": string  // Consider the climate and natural surroundings of the country. Determine if you prefer a warm or cold climate, proximity to the beach, mountains, or other natural attractions.
	"safety": string  // Research the country's safety record, crime rates, and political stability. Consider how safe you will feel living there and if the country has a stable legal system

In [59]:
template_string42 = """Give me reasons why {country} is a good place to retire: 

Cost of living, Healthcare, Climate, Safety, Lifestyle, and Visa. 

Write in {language}. 

{format_instructions}
"""


In [60]:
prompt_template42 = ChatPromptTemplate.from_template(template_string42)

In [61]:
country_to_retire = "Nepal"   # Not Japan
language_to_write = "Funny English"

prompt42 = prompt_template42.format_messages(
                    country=country_to_retire,
                    language=language_to_write, 
                    format_instructions=format_instructions)

In [62]:
print(prompt42[0].content)

Give me reasons why Nepal is a good place to retire: 

Cost of living, Healthcare, Climate, Safety, Lifestyle, and Visa. 

Write in Funny English. 

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"cost_of_living": string  // Assess the cost of housing, healthcare, groceries, transportation, and other essential expenses in the country you are considering. Ensure that your retirement income or savings can adequately cover your desired lifestyle.
	"healthcare": string  // Look into the quality and accessibility of healthcare services in the country. Consider the availability of affordable healthcare insurance or the country's public healthcare system.
	"climate": string  // Consider the climate and natural surroundings of the country. Determine if you prefer a warm or cold climate, proximity to the beach, mountains, or other natural attractions.
	"safety": string  // Research the cou

In [63]:
reasons42 = chat(prompt42)

In [64]:
print(reasons42.content)

```json
{
	"cost_of_living": "Nepal is cheap like momo, you can live like a king with your retirement savings. You can buy a whole goat for the price of a Starbucks coffee.",
	"healthcare": "Don't worry about your health, Nepal has Ayurvedic medicine that can cure anything from a headache to a broken heart. Plus, the fresh mountain air will do wonders for your lungs.",
	"climate": "Nepal has the perfect climate for retirees who want to escape the heat or the cold. You can enjoy the cool mountain breeze in the summer and snuggle up by the fire in the winter.",
	"safety": "Nepal is safer than your grandma's house. The only danger you'll face is getting lost in the beauty of the Himalayas.",
	"lifestyle": "Nepal has a laid-back lifestyle that will make you forget about the hustle and bustle of city life. You can spend your days hiking, meditating, or just enjoying a cup of chai with the locals.",
	"visa": "Getting a visa in Nepal is easier than making a cup of instant noodles. You can get

In [65]:
print(type(reasons42.content))

<class 'str'>


In [66]:
output_dict = output_parser.parse(reasons42.content)

In [67]:
type(output_dict)

dict

In [72]:
from pprint import pprint

In [73]:
pprint(output_dict)

{'climate': 'Nepal has the perfect climate for retirees who want to escape the '
            'heat or the cold. You can enjoy the cool mountain breeze in the '
            'summer and snuggle up by the fire in the winter.',
 'cost_of_living': 'Nepal is cheap like momo, you can live like a king with '
                   'your retirement savings. You can buy a whole goat for the '
                   'price of a Starbucks coffee.',
 'healthcare': "Don't worry about your health, Nepal has Ayurvedic medicine "
               'that can cure anything from a headache to a broken heart. '
               'Plus, the fresh mountain air will do wonders for your lungs.',
 'lifestyle': 'Nepal has a laid-back lifestyle that will make you forget about '
              'the hustle and bustle of city life. You can spend your days '
              'hiking, meditating, or just enjoying a cup of chai with the '
              'locals.',
 'safety': "Nepal is safer than your grandma's house. The only danger you'

In [74]:
print(output_dict["visa"])

Getting a visa in Nepal is easier than making a cup of instant noodles. You can get a retirement visa in no time and start living your dream life in the land of the Gurkhas.
